<a href="https://colab.research.google.com/github/JunhyeongTPark/eeg_vae/blob/main/PD_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

Load libraries, set up deterministic algoirthms and seed for reproducibility, initialize functions. Much of the setup for this file is the same as the PD_Multiband_VAE file as we need to load in the model we trained.

##Configurations

In [1]:
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [2]:
!pip install pytictoc

In [3]:
from pytictoc import TicToc

import numpy as np
import pandas as pd
import os
import time
import random

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
import math
from torch.optim.lr_scheduler import ReduceLROnPlateau
from google.colab import runtime, drive
from torch.utils.checkpoint import checkpoint
import itertools

In [4]:
if torch.cuda.is_available():
    print("GPU is available and being used.")
else:
    print("GPU is not available or not being used.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU is available and being used.


In [5]:
print("Mounting Google Drive at /content/drive")
drive.mount('/content/drive')

Mounting Google Drive at /content/drive
Mounted at /content/drive


In [6]:
def seed_everything(seed=42):
    # Set seed for RNGs
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    # Use deterministic algorithms
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)

SEED = 42
seed_everything(SEED)

In [7]:
# To gauge training duration
t = TicToc()

##Load Data

In [8]:
processed_data = torch.load('/content/drive/MyDrive/Colab_Notebooks/processed_data_cz.pt')

In [9]:
train_signals = processed_data['train_signals']
train_labels = processed_data['train_labels']
val_signals = processed_data['val_signals']
val_labels = processed_data['val_labels']
test_signals = processed_data['test_signals']
test_labels = processed_data['test_labels']

In [10]:
# Verifying class proportions
trl, train_counts = torch.unique(train_labels, return_counts=True)
val_trl, val_counts = torch.unique(val_labels, return_counts=True)
test_trl, test_counts = torch.unique(test_labels, return_counts=True)

print(train_counts / len(train_labels))
print(val_counts / len(val_labels))
print(test_counts / len(test_labels))

tensor([0.3269, 0.3240, 0.2204, 0.1287])
tensor([0.3264, 0.3243, 0.2203, 0.1290])
tensor([0.3273, 0.3235, 0.2204, 0.1289])


In [11]:
train_signals.shape

torch.Size([17349, 4, 1, 251])

##Dataset & DataLoader

In [12]:
class EEGDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [13]:
def seed_worker(worker_id):
    # Seed dataloader workers for reproducibility
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g_loader = torch.Generator()
g_loader.manual_seed(SEED)

In [14]:
train_data = EEGDataset(train_signals, train_labels)
val_data = EEGDataset(val_signals, val_labels)
test_data = EEGDataset(test_signals, test_labels)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True,
    num_workers=4, pin_memory=True, worker_init_fn=seed_worker, generator=g_loader)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True,
    num_workers=4,pin_memory=True, worker_init_fn=seed_worker, generator=g_loader)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True,
    num_workers=4, pin_memory=True, worker_init_fn=seed_worker, generator=g_loader)

##Load Model

In [15]:
def load_model(BANDS, model_hparams, checkpoint_name, folder='SM'):
    multiband_model = MultiBandVae(hparams=model_hparams, num_bands=len(BANDS)).to(device)

    if folder == 'TM':
        folder_name = '/content/drive/My Drive/Colab_Notebooks/Saved Models/Transformer Models/'
    elif folder == 'FT':
        folder_name = '/content/drive/My Drive/Colab_Notebooks/Saved Models/Fine Tuning/'
    elif folder == 'FC':
        folder_name = "/content/drive/My Drive/Colab_Notebooks/Saved Models/Final Candidates/"
    else:
        folder_name = '/content/drive/My Drive/Colab_Notebooks/Saved Models/'
    filename = checkpoint_name + '.pth'

    PATH = os.path.join(folder_name, filename)

    if os.path.exists(PATH):
        print(f'Loading model checkpoint from {PATH}')

        checkpoint = torch.load(PATH, map_location=device)
        multiband_model.load_state_dict(checkpoint['model_state_dict'])
        last_epoch = checkpoint['epoch']
        train_losses = checkpoint['train_losses']
        val_losses = checkpoint['val_losses']
        # checkpoint_hparams = checkpoint.get('hyperparameters', None)

        print(f'Loaded checkpoint from epoch {last_epoch}')
        return multiband_model, last_epoch, train_losses, val_losses #, checkpoint_hparams
    else:
        print(f'File not found at {PATH} - Check your folder name')
        return None, None, None, None

##Save & Load SGL Model

In [20]:
def save_sgl_model(model, drive_path):
    print('Saving best model')
    os.makedirs(os.path.dirname(drive_path), exist_ok=True)
    torch.save(model.state_dict(), drive_path)

    print('Model saved successfully')

def load_sgl_model(drive_path, n_features, n_classes, device='cuda'):
    print('Loading model')
    model = MultinomialSGL(n_features, n_classes).to(device)
    model.load_state_dict(torch.load(drive_path)).to(device)
    model.eval()
    print('Model loaded successfully')
    return model

##VAE Models

In [16]:
# Same Transformer-based VAE from PD-Multiband-VAE
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=251):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout, activation='gelu'):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.activation = F.gelu if activation == 'gelu' else F.relu

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        x = src
        x_norm = self.norm1(x)
        attn_output, _ = self.self_attn(x_norm, x_norm, x_norm, attn_mask=src_mask)
        x = x + self.dropout1(attn_output)

        x_res = x
        x_norm = self.norm2(x_res)
        ff_output = self.linear2(self.dropout(self.activation(self.linear1(x_norm))))
        x = x_res + self.dropout2(ff_output)
        return x

class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout, activation='gelu'):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.multihead_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

        self. activation = F.gelu if activation == 'gelu' else F.relu

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None,
                tgt_key_padding_mask=None, memory_key_padding_mask=None):
        x = tgt
        x_norm = self.norm1(x)
        attn_output, _ = self.self_attn(x_norm, x_norm, x_norm, attn_mask=tgt_mask)
        x = x + self.dropout1(attn_output)

        x_res = x
        x_norm = self.norm2(x_res)
        attn_output, _ = self.multihead_attn(x_norm, memory, memory)
        x = x_res + self.dropout2(attn_output)

        x_res = x
        x_norm = self.norm3(x_res)
        ff_output = self.linear2(self.dropout(self.activation(self.linear1(x_norm))))
        x = x_res + self.dropout3(ff_output)
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, num_electrodes=8, seq_len=251, d_model=128, nhead=8, num_layers=3, latent_dim=64, dropout=0.2, device='cuda'):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model

        self.mask = nn.Transformer.generate_square_subsequent_mask(self.seq_len).to(device)

        self.conv_stem = nn.Sequential(
            # [B, C, T] --> [B, d_model, T]
            nn.Conv1d(in_channels=num_electrodes, out_channels=d_model // 4, kernel_size=7, padding=3),
            nn.BatchNorm1d(d_model // 4),
            nn.GELU(),
            nn.Conv1d(in_channels=d_model // 4, out_channels=d_model // 2, kernel_size=5, padding=2),
            nn.BatchNorm1d(d_model // 2),
            nn.GELU(),
            nn.Conv1d(in_channels=d_model // 2, out_channels=d_model, kernel_size=3, padding=1),
            nn.BatchNorm1d(d_model),
            nn.GELU()
        )

        self.pos_encoder = PositionalEncoding(d_model, max_len=seq_len)

        self.layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, nhead, d_model*4, dropout) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)

        self.fc_mu = nn.Linear(d_model, latent_dim)
        self.fc_logvar = nn.Linear(d_model, latent_dim)

    def forward(self, x):
        x = self.conv_stem(x)

        # x shape: [B, C, T] -> permute to [B, T, C]
        x = x.permute(0, 2, 1)

        # Embed input and add positional encoding
        x = self.pos_encoder(x)

        # Pass through Transformer encoder
        for layer in self.layers:
            x = layer(x, src_mask=self.mask)
        x = self.norm(x)

        # Use last time step
        sequence_embedding = x[:, -1, :]
        mu = self.fc_mu(sequence_embedding)
        logvar = self.fc_logvar(sequence_embedding)

        return mu, logvar

class TransformerDecoder(nn.Module):
    def __init__(self, num_electrodes=8, seq_len=251, d_model=128, nhead=8, num_layers=3, latent_dim=64, dropout=0.2):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model

        # Project Latent Vector to Sequence
        self.latent_to_seq = nn.Linear(latent_dim, d_model)

        # Create a "Query" sequence for the decoder
        self.decoder_query = nn.Parameter(torch.randn(1, seq_len, d_model))

        # Transformer Decoder
        self.layers = nn.ModuleList([
            TransformerDecoderLayer(d_model, nhead, d_model*4, dropout) for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)

        # Output Projection
        self.fc_out = nn.Linear(d_model, num_electrodes)

    def forward(self, z):
        batch_size = z.size(0)

        memory = self.latent_to_seq(z).unsqueeze(1)
        tgt = self.decoder_query.repeat(batch_size, 1, 1)

        for layer in self.layers:
            tgt = layer(tgt, memory)
        output = self.norm(tgt)

        output = self.fc_out(output)

        return output.permute(0, 2, 1)

class MultiBandVae(nn.Module):
    def __init__(self, num_bands=5,
                 hparams = {
                      'num_electrodes': 1,
                      'seq_len': 251,
                      'd_model': 256,
                      'nhead': 8,
                      'num_layers': 3,
                      'latent_dim': 32,
                      'dropout': 0.2
                  }
        ):
        super().__init__()
        self.num_bands = num_bands
        self.latent_dim = hparams['latent_dim']

        base_hparams = hparams.copy()

        alpha_hparams = hparams.copy()
        alpha_hparams.update({
            'd_model': 512,
            'nhead': 16,
            'num_layers': 3,
            'latent_dim': int(self.latent_dim * 1.5)
        })

        beta_hparams = hparams.copy()
        beta_hparams.update({
            'd_model': 768,
            'nhead': 16,
            'num_layers': 4,
            'latent_dim': int(self.latent_dim * 2)
        })

        all_hparams = [base_hparams, base_hparams, alpha_hparams, beta_hparams]

        self.encoders = nn.ModuleList([
            TransformerEncoder(**base_hparams),
            TransformerEncoder(**base_hparams),
            TransformerEncoder(**alpha_hparams),
            TransformerEncoder(**beta_hparams)
        ])

        self.decoders = nn.ModuleList([
            TransformerDecoder(**base_hparams),
            TransformerDecoder(**base_hparams),
            TransformerDecoder(**alpha_hparams),
            TransformerDecoder(**beta_hparams)
        ])

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def encode(self, x):
        mus, logvars, zs = [], [], []

        for i in range(self.num_bands):
            band_x = x[:, i, :, :]

            mu, logvar = self.encoders[i](band_x)

            mus.append(mu)
            logvars.append(logvar)

            z = self.reparameterize(mu, logvar)

            zs.append(z)

        return mus, logvars, zs

    def decode(self, zs):
        recon_bands = []
        for i in range(self.num_bands):
            recon_band = self.decoders[i](zs[i])
            recon_bands.append(recon_band)

        return torch.stack(recon_bands, dim=1)

    def forward(self, x):
        mus, logvars, zs = self.encode(x)
        recon_combined = self.decode(zs)

        return recon_combined, mus, logvars

    def generate(self, n_samples=1, temperature=1.0, device='cuda'):
        self.eval()

        zs = []
        for i in range(self.num_bands):
            latent_dim = self.encoders[i].fc_mu.out_features
            zs.append(torch.randn(n_samples, latent_dim, device=device) * temperature)

        with torch.no_grad():
            return self.decode(zs)

#Classification

##Load VAE Model

###Transformer-VAE Time Frequency Model

In [17]:
BANDS = ['delta', 'theta', 'alpha', 'beta']
model_hparams = {
    'num_electrodes': 1,
    'seq_len': 251,
    'd_model': 256,
    'nhead': 8,
    'num_layers': 3,
    'latent_dim': 32,
    'dropout': 0.2
}

# Load Model
multiband_model, last_epoch, train_losses, val_losses = load_model(BANDS, model_hparams, 'Model_E489', 'FC')

Loading model checkpoint from /content/drive/My Drive/Colab_Notebooks/Saved Models/Final Candidates/Model_E489.pth
Loaded checkpoint from epoch 489


##Data Preparation

In [19]:
def extract_features(model, data_loader, device='cuda'):
    model.eval()
    all_features = []
    all_labels = []

    with torch.no_grad():
        for x, labels in data_loader:
            x = x.to(device)
            # Extract mu vector from VAE to use as representation
            _, mus, _ = model(x)
            batch_features = np.concatenate([mu.cpu().numpy() for mu in mus], axis=1)
            all_features.append(batch_features)
            all_labels.append(labels.cpu().numpy())

    X = np.concatenate(all_features, axis=0)
    y = np.concatenate(all_labels, axis=0)

    return X, y

X_train, y_train = extract_features(multiband_model, train_loader)
X_val, y_val = extract_features(multiband_model, val_loader)
X_test, y_test = extract_features(multiband_model, test_loader)

print('Final Shapes:')
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'X_val: {X_val.shape}, y_val: {y_val.shape}')
print(f'X_test: {X_test.shape}, y_test: {y_test.shape}')

Final Shapes:
X_train: (17349, 176), y_train: (17349,)
X_val: (8674, 176), y_val: (8674,)
X_test: (8675, 176), y_test: (8675,)


In [21]:
# Function to get data for specific classes only
def extract_class_data(X, y, class_labels):
    X = np.array(X)
    y = np.array(y)
    class_data = []
    labels = []
    for label in class_labels:
        class_indices = np.where(y == label)[0]
        class_data.append(X[class_indices])
        labels.append(y[class_indices])
    return np.concatenate(class_data, axis=0), np.concatenate(labels, axis=0)

In [ ]:
# Data with only PDMCI and PDD patient data
X_MCI_D_train, y_MCI_D_train = extract_class_data(X_train, y_train, [2,3])
X_MCI_D_val, y_MCI_D_val = extract_class_data(X_train, y_train, [2,3])
X_MCI_D_test, y_MCI_D_test = extract_class_data(X_test, y_test, [2,3])

In [ ]:
train_signals = train_signals.reshape(train_signals.shape[0],-1)
val_signals = val_signals.reshape(val_signals.shape[0],-1)
test_signals = test_signals.reshape(test_signals.shape[0],-1)

# Raw data to test performance of latent vector in classificaiton vs raw data
X_raw_MCI_D_train, y_raw_MCI_D_train = extract_class_data(train_signals, train_labels, [2,3])
X_raw_MCI_D_val, y_raw_MCI_D_val = extract_class_data(val_signals, val_labels, [2,3])
X_raw_MCI_D_test, y_raw_MCI_D_test = extract_class_data(test_signals, test_labels, [2,3])

##SGL Setup

MultinomialSGL is a Sparse Group Lasso model. Libraries like `sklean` use one vs rest models for multi-label classification, so a shallow neural network was used instead with the use of a custom loss function.

In [22]:
class MultinomialSGL(nn.Module):
    def __init__(self, n_features, n_classes):
        super().__init__()
        self.linear = nn.Linear(n_features, n_classes)

    def forward(self, x):
        return self.linear(x)

def sgl_loss(logits, y_true, model, groups, l1_reg, group_reg, class_weights=None):
    cross_entropy_loss = nn.CrossEntropyLoss(weight=class_weights)(logits, y_true)

    weights = model.linear.weight # [n_classes, n_features]
    l1_penalty = l1_reg * torch.norm(weights, p =1)

    group_penalty = 0.0
    unique_groups = torch.unique(groups)

    for group in unique_groups:
        group_mask = (groups == group)
        group_weights = weights[:, group_mask]
        group_penalty += torch.norm(group_weights, p=2)

    group_penalty *= group_reg

    total_loss = cross_entropy_loss + l1_penalty + group_penalty

    return total_loss

In [23]:
def cv_sgl(X_train, y_train, X_val, y_val, groups, l1_reg, group_reg, n_classes, epochs=100, lr=1e-3):
    device = X_train.device
    n_features = X_train.shape[1]

    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,
        num_workers=4, pin_memory=True, worker_init_fn=seed_worker, generator=g_loader)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False,
        num_workers=4, pin_memory=True, worker_init_fn=seed_worker, generator=g_loader)

    # Class weights due to disproportionate classes in dataset
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train.cpu()), y=y_train.cpu().numpy())
    class_weights = torch.tensor(class_weights, dtype=torch.float32, device=device)

    model = MultinomialSGL(n_features, n_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        for X, y in train_loader:
            optimizer.zero_grad()
            logits = model(X)
            loss = sgl_loss(logits, y, model, groups, l1_reg, group_reg, class_weights)
            loss.backward()
            optimizer.step()

    model.eval()
    all_preds = []
    with torch.no_grad():
        for X, _ in val_loader:
            logits = model(X)
            preds = torch.argmax(logits, dim=1)
            all_preds.append(preds)

    all_preds = torch.cat(all_preds, dim=0)
    val_accuracy = accuracy_score(y_val.cpu().numpy(), all_preds.cpu().numpy())

    return val_accuracy, model

##SGL

In [25]:
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).long()
X_val_tensor = torch.from_numpy(X_val).float()
y_val_tensor = torch.from_numpy(y_val).long()
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).long()

In [26]:
groups_list = []
groups_list.extend([0] * 32)  # delta
groups_list.extend([1] * 32)  # theta
groups_list.extend([2] * 48)  # alpha
groups_list.extend([3] * 64)  # beta
groups = torch.tensor(groups_list)
n_features = groups.shape[0]

In [27]:
# param_grid = {
#     'l1_reg': np.logspace(-4, 1, 10),
#     'group_reg': np.logspace(-4, 1, 10)
# }
param_grid = {
    'l1_reg': np.logspace(-4, 1, 2),
    'group_reg': np.logspace(-4, 1, 2)
}

best_score = -1
best_params = {}
best_sgl_model = None

param_combinations = list(itertools.product(param_grid['l1_reg'], param_grid['group_reg']))

for l1_reg, group_reg in tqdm(param_combinations, desc='Grid Searching'):
    score, model = cv_sgl(X_train_tensor, y_train_tensor,
                          X_val_tensor, y_val_tensor, groups,
                          l1_reg, group_reg, n_classes=4)

    if score > best_score:
        best_score = score
        best_params = {'l1_reg': l1_reg, 'group_reg': group_reg}
        best_sgl_model = model


print(f"Best Score: {best_score:.4f}")
print(f"Best Parameters: {best_params}")

if best_sgl_model:
    coefs = best_sgl_model.linear.weight.detach().cpu().numpy()

    print(f'\nShape of coefficients: {coefs.shape}')

    non_zero_mask = np.abs(coefs).sum(axis=0) > 1e-6
    num_non_zero = np.sum(non_zero_mask)
    print(f'Number of non-zero features: {num_non_zero} out of {n_features}')

    model_save_path = '/content/drive/My Drive/Colab_Notebooks/Saved Models/best_sgl_classifier.pth'
    save_sgl_model(best_sgl_model, model_save_path)

Grid Searching: 100%|██████████| 4/4 [10:06<00:00, 151.64s/it]

Best Score: 0.4211
Best Parameters: {'l1_reg': np.float64(0.0001), 'group_reg': np.float64(0.0001)}

Shape of coefficients: (4, 176)
Number of non-zero features: 176 out of 176
Saving best model
Model saved successfully


In [31]:
# best_sgl_model = load_sgl_model(model_save_path, n_features, 4, device)

with torch.no_grad():
    logits = best_sgl_model(X_test_tensor)
    predictions = torch.argmax(logits, dim=1)
    accuracy = accuracy_score(y_test_tensor.cpu().numpy(), predictions.cpu().numpy())
    print(f'Validation Accuracy: {accuracy:.4f}')

Validation Accuracy: 0.4236


In [32]:
runtime.unassign()